# Script para convertir bounding boxes a formato YOLO v3
Se basa en la documentación aportada por: https://medium.com/@manivannan_data/how-to-train-yolov3-to-detect-custom-objects-ccbcafeb13d2

In [1]:
import cv2
import numpy as np

import matplotlib.pyplot as plt

In [2]:
import sys 
import os

sys.path.append(os.path.abspath("../"))

In [3]:
def boundToYoloBound(class_number, bound, img_width, img_height):
    corner_a = bound[0:2]
    corner_b = bound[2:4]
    
    width = abs(corner_a[0]-corner_b[0])
    height = abs(corner_a[1]-corner_b[1])
    
    x = (width/2) + min(corner_a[0], corner_b[0])
    y = (height/2) + min(corner_a[1], corner_b[1])
    
    width /= img_width
    height /= img_height
    
    x /= img_width
    y /= img_height    
    
    #<object-class> <x> <y> <width> <height>
    #<class_number> (<absolute_x> / <image_width>) (<absolute_y> / <image_height>)
    #(<absolute_width> / <image_width>) (<absolute_height> / <image_height>)
    
    return [class_number, x, y, width, height]

In [5]:
assert boundToYoloBound(0, [5, 4, 2, 2], 8, 8) == [0, 0.4375, 0.375, 0.375, 0.25]
assert boundToYoloBound(0, [5, 4, 2, 2], 8, 8) == boundToYoloBound(0, [2, 2, 5, 4], 8, 8)

In [6]:
dataset_folder = "YOLOv3Dataset_"

In [11]:
import shutil
import os

shutil.rmtree(dataset_folder)
os.mkdir(dataset_folder)
os.mkdir(dataset_folder + "/dataset")

In [12]:
import csv
import lib

file_name = '../Etiquetado/bounds2.txt'
bounds_dict, _ = lib.Utils.openBounds(file_name)

image_scaling = 0.5
class_number = 0

cap = cv2.VideoCapture('../DCIM/saturated-noaudio.avi')
racap = lib.RACapWrapper(cap, 30)

height, width = None, None

for frame_n in sorted(bounds_dict.keys()):    
    frame, _ = racap.readAt(frame_n)
    
    if height is None:
        [height, width, _] = frame.shape
    
    frame = cv2.resize(frame, None, fx=image_scaling, fy=image_scaling, interpolation=cv2.INTER_NEAREST)
    
    cv2.imwrite(dataset_folder + "/dataset/" + str(frame_n) + ".jpg", frame)
    
    with open(dataset_folder + "/dataset/" + str(frame_n) + ".txt", "w", newline='') as bounds_file:    
        bound_writer = csv.writer(bounds_file, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
        for bound in bounds_dict[frame_n]:
            bound_writer.writerow(boundToYoloBound(class_number, bound, width, height))   
            
cap.release()

In [13]:
import os

def saveFileList(file_name, file_list):
     with open(file_name, "w") as file:    
        for file_name in file_list:
            file.write(file_name + "\n")

dataset_path = "/home/lbarrera/Documentos/proyecto_ia/YOLO/Dataset/"
percentage_train = 0.7;

for r, d, files in os.walk(dataset_folder + "/dataset"):
    files = [f for f in files if ".jpg" in f]
    
    frames_files = len(files)
    train_files_count = int(frames_files*percentage_train)
    
    saveFileList(dataset_folder + "/train.txt", [dataset_path + f for f in files[0:train_files_count]])
    saveFileList(dataset_folder + "/test.txt", [dataset_path + f for f in files[train_files_count:]])